In [1]:
# -*- coding: utf-8 -*-
# 从数据库中将爬取的职位信息取出来，将和推荐相关的要素，编成一个职位的特征码，再把特征码解成只有 0-1 的各个职位特征矩阵，存起来
# 这个可以离线执行
import pandas as pd
import re

In [2]:
# 存一下，避免以后仍然需要开数据库，数据库量大时就不再执行这段了
#df.to_csv('data.csv', index=False)
df = pd.DataFrame(pd.read_csv(r'./data.csv'))

In [3]:
# 从数据库中取出要生成编码标签的素材，直接用 df 也行，只是用来排除无关列
wait_to_encoded = df.loc[:,['id','position_name','city','education','position_requirements','create_time']]

In [4]:
number    = list(range(0,11))
# 这里源自国际人的就业质量报告，分类标准比以往数据库内的更好，不容易重叠，后期可添加
type_name = ['其他岗位','管培生岗位','数据算法','开发测试运维','游戏类岗位','售前售后技术支持','金融财务','互联网运营','互联网产品','营销销售岗','制造业岗位']
type_list = dict(zip(type_name,number))
# 后期可以添加，最多99个，这里暂时先放常用的
city_name = ['其他','北京','上海','广州','深圳','杭州','武汉','成都','贵阳','大连','沈阳']
city_list = dict(zip(city_name,number))
education_list = {'其他':0,'本科':1,'硕士':2,'大专':3}
intern_orfull = {'实习':1,'全职':2}
time_type ={'近一周':1,'一周到一个月':2,'一个月以上':3}

In [5]:
# 这里后期会将爬取的职位名称进行解词，做词频统计，词频高的放入此处，代码示例在最下方
# 或由行业专家定义这里的词，暂时先放些我个人自定义的词
# 主要是为了省掉人工标注的过程
type_words = []
type_words.append([])
type_words.append(['管理培训','管培'])
type_words.append(['数据','算法','可视化','自然语言','机器学习','NLP','nlp'])
type_words.append(['JAVA','C','开发','工程师'])
type_words.append(['游戏'])
type_words.append(['售前','售后'])
type_words.append(['财务','会计','投资','债券','财经'])
type_words.append(['运营'])
type_words.append(['产品'])
type_words.append(['销售','中教','顾问'])
type_words.append(['制造'])

In [6]:
# 编码阶段
code = []
for indexs in wait_to_encoded.index:
    name = wait_to_encoded.loc[indexs].values[1]
    educ = wait_to_encoded.loc[indexs].values[3]
    city = wait_to_encoded.loc[indexs].values[2]
    time = wait_to_encoded.loc[indexs].values[5]
    # 1     实习/全职
    if re.search('实习',name) != None:
        code.append(1 * intern_orfull['实习'])
    else:
        code.append(1 * intern_orfull['全职'])
    
    # 10    学历要求
    for key,value in education_list.items():
        if re.search(key,educ) != None:
            code[indexs] += 10 * value
    
    # 1100   城市
    for key,value in city_list.items():
        if re.search(key,city) != None:
            code[indexs] += 100 * value
            break                          # 多个城市只选顺序第一个找到的
    
    # 110000 职位类型
    for key,value in type_list.items():
        flag = 0
        for index in type_words[value]:
            if re.search(index,name) != None:
                flag = 1                   # 在职位类型排序时考虑了顺序，这里做单一分类，并按优先顺序找
        if flag == 1:
            code[indexs] += 10000 * type_list[key] 
            break
    
    # 1000000 判断职位时间的远近，暂时分三级 ，越近的数越小，分级标准为暂定，这里全是1
    # 待时间格式化后，再做整体修改
    code[indexs] += 1000000 * 1
    
    # 以后可能会在更高的数位上放关于 JD 里能力的标签，先不做

In [7]:
# 存编码，这里暂时存 csv 里，后期连数据库在数据库中新建职位 id 与编码对应的职位特征表
df['code'] = code

In [8]:
coder = pd.DataFrame(
    {
        'id' : [i for item in (df[['id']].values.tolist()) for i in item],
        'code' : [i for item in (df[['code']].values.tolist()) for i in item],
    }
)

coder = coder.reindex(columns=['id','code','近一周','一周到一个月','一个月以上','其他岗位','管培生岗位','数据算法','开发测试运维','游戏类岗位','售前售后技术支持','金融财务','互联网运营','互联网产品','营销销售岗','制造业岗位','其他','北京','上海','广州','深圳','杭州','武汉','成都','贵阳','大连','沈阳','其他','本科','硕士','大专','实习','全职'])

In [9]:
# 解码成 0-1 矩阵
pd.set_option('display.max_columns', None)
for indexs in coder.index:
    code_str = str(int(coder.loc[indexs].values[1]))
    for key,value in time_type.items():
        if value == int(code_str[0]):
            coder.loc[indexs,key] = 1
        else:
            coder.loc[indexs,key] = 0
    for key,value in type_list.items():
        if value == int(code_str[1]+code_str[2]):
             coder.loc[indexs,key] = 1
        else:
            coder.loc[indexs,key] = 0
    for key,value in city_list.items():
        if value == int(code_str[3]+code_str[4]):
             coder.loc[indexs,key] = 1
        else:
            coder.loc[indexs,key] = 0
    for key,value in education_list.items():
        if value == int(code_str[5]):
             coder.loc[indexs,key] = 1
        else:
            coder.loc[indexs,key] = 0
    for key,value in intern_orfull.items():
        if value == int(code_str[6]):
             coder.loc[indexs,key] = 1
        else:
            coder.loc[indexs,key] = 0
    #print(code_str[1]+code_str[2])
coder.head()


,id,code,近一周,一周到一个月,一个月以上,其他岗位,管培生岗位,数据算法,开发测试运维,游戏类岗位,售前售后技术支持,金融财务,互联网运营,互联网产品,营销销售岗,制造业岗位,其他,北京,上海,广州,深圳,杭州,武汉,成都,贵阳,大连,沈阳,其他,本科,硕士,大专,实习,全职
0,1,1060112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,1070012,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,17,1080412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,3,1070412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,1020412,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [10]:
coder.to_csv('items_tags.csv', index=False)

In [ ]:
# 解词的准备代码
import jieba
text=''
name = list(df['position_name'])
JD = list(df['position_requirements'])
i = 0
for index in name:
    text+=name[i]
    i += 1
j = 0
for index in JD:
    text+=str(JD[j])
    j += 1
seg_list = jieba.lcut(text, cut_all=False)
stopwords_chinese = [line.strip() for line in open(r'C:\Users\Administrator\Desktop\stopwords_chinese.txt',encoding='UTF-8').readlines()] #文件下载地址：https://raw.githubusercontent.com/dongxiexidian/Chinese/master/stopwords.dat
word_list=[]
stopwords_chinese_add = ['\n',' ','  ','   ','    ','\r','\t','\r\n',]
for seg in seg_list:
    if seg not in stopwords_chinese:
        if seg !='\n'and seg !='   ' and  seg !='\r' and  seg !='\t':
            word_list.append(seg)
# print(stopwords_chinese)
word_list1=pd.DataFrame({'tag':word_list})
word_rank = pd.DataFrame(word_list1['tag'].value_counts())
pd.set_option('display.max_rows', None)


## 这段可以不运行